In [203]:
import numpy as np
import pandas as pd
import os.path
import subprocess
import math
import datetime
import matplotlib
import os.path
from copy import copy, deepcopy
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from datetime import datetime, timedelta

In [204]:
#settings
start_date=datetime(2017,12,6)
start_date_scen=datetime(2018,1,9)
num_days=start_date_scen-start_date
num_days=num_days.days
num_days_scen=14

# ---- scenario - INPUT / ADJUSTMENT / CALIBRATION
scenprec_corrfact=1.0              # Factor to correct precipitation in second week of scenario - normally strongly overestimated...

Need to manually remove all junk characters from forecast file before using!

In [205]:
dfs=pd.read_csv('../data/raw/forecast_2018.csv',encoding='latin-1',skiprows=1, sep=';')
df=pd.read_csv('../data/interim/weather.csv',parse_dates=['When'], index_col=0)
dfs

,year,month,day,2,5,8,11,14,17,20,23,Unnamed: 11,Unnamed: 12
0,2018,1.0,9.0,1.1,0,0.2,0.5,1.2,1.1,0.9,0.7,NaN,TEMPERATURE (deg C)
1,2018,1.0,9.0,1,7,5,1,-,1,4,1,NaN,PRECIPITATION (mm/3h)
2,2018,1.0,10.0,-1,-3.3,-5.3,-5.9,-5.2,-5.8,-7.7,-9.4,NaN,NaN
3,2018,1.0,10.0,-,-,-,-,-,-,-,-,NaN,NaN
4,2018,1.0,11.0,-10.2,-11.6,-12.7,-9.3,-2.9,-3.6,-11.4,-16,NaN,NaN
5,2018,1.0,11.0,-,-,-,-,-,-,-,-,NaN,NaN
6,2018,1.0,12.0,-15.2,-13.2,-13.6,-9.6,-3.6,-2.2,-4.8,-7,NaN,NaN
7,2018,1.0,12.0,-,-,-,-,-,-,-,-,NaN,NaN
8,2018,1.0,13.0,-6.9,-7.9,-10,-7.7,-2.8,-3.1,-10,-15.5,NaN,NaN
9,2018,1.0,13.0,-,-,-,-,-,-,-,-,NaN,NaN


In [206]:
#Clean file
dfs=dfs.drop('Unnamed: 11', axis=1)
df['Year']=df['When'].apply(lambda x:x.year)
df['Mo']=df['When'].apply(lambda x:x.month)
df['Day']=df['When'].apply(lambda x:x.day)

mask =(df['When'] < start_date_scen)
df=df.loc[mask]
df=df.reset_index(drop=True)
df.tail()

,When,Rad,Tair,RH,Wind,Pressure,Prec,Cl,ew,rho_air,huma,DoY,Year,Mo,Day
811,2018-01-08 19:00:00,0.0,-1.254,85.8,1.2,83020.0,0.0,0.56488,478.39521,1.06368,0.00360,8,2018,1,8
812,2018-01-08 20:00:00,0.0,2.646,62.8,3.8,83020.0,0.0,0.56488,463.86898,1.04864,0.00349,8,2018,1,8
813,2018-01-08 21:00:00,0.0,4.846,49.1,2.6,82990.0,0.0,0.56488,423.36901,1.03996,0.00319,8,2018,1,8
814,2018-01-08 22:00:00,0.0,2.646,61.0,2.8,82960.0,0.0,0.56488,450.57338,1.04788,0.00340,8,2018,1,8
815,2018-01-08 23:00:00,0.0,0.946,73.6,2.4,82930.0,0.0,0.56488,481.45391,1.05400,0.00363,8,2018,1,8


In [207]:
#Clean Scenario file
dfs1=dfs[0:14]  #WEEK1
i=1
dfp=pd.DataFrame({'A' : []})
while i<=dfs1.shape[0]:
    dfp=dfp.append(dfs1.loc[dfs1.index[i]])
    dfs1=dfs1.drop(dfs1.index[i],axis=0)
    i=i+1
dfp=dfp.drop('A',axis=1)
dfp=dfp[['day','2','5','8','11','14','17','20','23']]

dfs2=dfs[15:22]            #WEEK2
dfs2=dfs2.drop({'Unnamed: 12','14','17','20','23'}, axis=1)
dfs2.columns = ['year', 'month','day','Tmax','Tmin','Prec','Prec-percent']
dfs2=dfs2.reset_index(drop=True)
dfs2

,year,month,day,Tmax,Tmin,Prec,Prec-percent
0,2018,1.0,16.0,-4,-15,11,95
1,2018,1.0,17.0,-6,-12,10,100
2,2018,1.0,18.0,-5,-12,7,100
3,2018,1.0,19.0,-5,-10,6,95
4,2018,1.0,20.0,-5,-12,5,90
5,2018,1.0,21.0,-5,-12,4,75
6,2018,1.0,22.0,-5,-13,5,75


In [208]:
#Daily Temperature Ranges
dfs1= dfs1.convert_objects(convert_numeric=True)
for index,row in dfs1.iterrows():
    dfs1.loc[index,'Trange']=dfs1.loc[index,['2','5','8','11','14','17','20','23']].max()-dfs1.loc[index,['2','5','8','11','14','17','20','23']].min()

dfs2= dfs2.convert_objects(convert_numeric=True)
dfs2['Trange']=dfs2['Tmax']-dfs2['Tmin']

l=[]
dft=pd.DataFrame({'A' : []})
dff=pd.DataFrame({'A' : []})
for i in range(0,num_days):
    run=True
    k=0
    while run:
        l.append(df.loc[int(24*i+k),'Tair'])

        k=k+1
        if k==24:
            run=False
    l=np.array(l)
    dft.loc[i,'Tr']=l.max()-l.min()
    dft.loc[i,'When']=df.loc[int(24*i),'When']
    l=[]

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  from ipykernel import kernelapp as app
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


In [209]:
#Select Climate variables using daily temperature range matches
j=0
for index,row in dfs1.iterrows():
    b=0
    l=np.array([])
    run=True
    l=abs(dfs1.loc[index,'Trange']-dft['Tr'])
    c=l.min()

    h=np.where(l==c)
    h=h[0]
    dff=dff.append(dft.loc[h[h.size-1]])
    j=j+1
    
j=0
for index,row in dfs2.iterrows():
    b=0
    l=np.array([])
    run=True
    l=abs(dfs2.loc[index,'Trange']-dft['Tr'])
    c=l.min()

    h=np.where(l==c)
    h=h[0]
    dff=dff.append(dft.loc[h[h.size-1]])
    j=j+1
dff=dff.drop('A',axis=1)
dff=dff.reset_index()
dft=dft.reset_index()

dft['When'] = pd.to_datetime(dft['When'])
dft['Date']= dft['When'].apply(lambda x:x.date())
dff['When'] = pd.to_datetime(dff['When'])
dff['Day']= dff['When'].apply(lambda x:x.day)

In [210]:
#Create scenario files
dfg=pd.DataFrame({'A' : []})
c=0
for index,row in dff.iterrows():
    l=np.where(df['Day']==dff['Day'][index])
    l=l[0]
    j=0
    while j<24:
        dfg=dfg.append(df.loc[l[0]+j],ignore_index=True)
        dfg.loc[c,'Date']=start_date_scen+timedelta(days=int(index),hours=j)
        j=j+1
        c=c+1

dfg=dfg.drop('A',axis=1)
dfg=dfg.reset_index()
dfp=dfp.reset_index()

In [211]:
dfp=dfp.replace('-', 0)
dfp=dfp.replace('1', 0.5)
dfp=dfp.drop('index',axis=1)
dfp=dfp.astype(float)

dfp2=pd.DataFrame({'A' : []})
dfs2['prec_sc']=dfs2['Prec']*dfs2['Prec-percent']/100*scenprec_corrfact
dfp2['day']=dfs2['day']
dfp2['prec']=dfs2['prec_sc']
dfp2=dfp2.drop('A',axis=1)
dfp2['prec']=dfp2['prec'].astype(float)

In [212]:
temp_dist=[0.2,0,0.1,0.4,0.8,1,0.7,0.4,0.3]
for i in range(0,7):
    c=0
    for j in range(2,24,3):
        dfs2.loc[i,str(j)]=temp_dist[c]*dfs2.loc[i,'Trange']+dfs2.loc[i,'Tmin']
        c+=1
dfs2=dfs2[['year','month','day','2','5','8','11','14','17','20','23','Trange']]

In [213]:
#Use forecast WEEK1
j=0
for i in range(0,7):
    l=np.where(dfs1['day']==dfg.loc[j,'Date'].day)
    k=0
    c=0
    if len(l[0])>0:
        while c!=24:
            dfg.loc[j+c,'Tair']=dfs1.iloc[l[0][0],3+k]
            dfg.loc[j+c,'Prec']=dfp.iloc[l[0][0],1+k]/3
            c=c+1
            if c%3==0:
                k=k+1

    j=j+24

#Use forecast WEEK2
for i in range(0,7):
    l=np.where(dfs2['day']==dfg.loc[j,'Date'].day)
    k=0
    c=0
    if len(l[0])>0:
        while c!=24:
            dfg.loc[j+c,'Tair']=dfs2.iloc[l[0][0],3+k]
            dfg.loc[j+c,'Prec']=dfp2.loc[l[0][0],'prec']/24
            c=c+1
            if c%3==0:
                k=k+1

    j=j+24

In [214]:
dfg['Prec']=(dfg['Prec'])/1000
dfg=dfg.round(7)
dfg['Date'] = pd.to_datetime(dfg['Date'])

In [215]:
#Add Original Weather Data
df['Time'],df['Date']= df['When'].apply(lambda x:x.time()), df['When'].apply(lambda x:x.date())
dfg['Time'],dfg['Date']= dfg['Date'].apply(lambda x:x.time()), dfg['Date'].apply(lambda x:x.date())
dfg = dfg[['Rad', 'Tair', 'RH', 'Wind','Pressure','Prec','Cl','Date','Time','huma','rho_air']]
df = df[['Rad', 'Tair', 'RH', 'Wind','Pressure','Prec','Cl','Date','Time','huma','rho_air']]

In [216]:
#Join AWS data and forecast
df=pd.concat([pd.DataFrame(df), dfg], ignore_index=True)
df = df[['Rad', 'Tair', 'RH', 'Wind','Pressure','Prec','Cl','Date','Time','huma','rho_air']]
df=df.round(7)

In [217]:
# #Experiments
# df['Tair']+=8
# df['Prec']+=0

In [218]:
#Output
df.to_csv('../data/interim/scenario.csv')
df.tail()

,Rad,Tair,RH,Wind,Pressure,Prec,Cl,Date,Time,huma,rho_air
1147,0.0,-7.4,78.8,1.3,82610.0,0.000156,0.40335,2018-01-22,19:00:00,0.00158,1.09758
1148,0.0,-7.4,75.1,1.4,82660.0,0.000156,0.40335,2018-01-22,20:00:00,0.00168,1.09241
1149,0.0,-9.8,75.6,0.6,82630.0,0.000156,0.40335,2018-01-22,21:00:00,0.00178,1.08954
1150,0.0,-9.8,80.2,0.8,82590.0,0.000156,0.40335,2018-01-22,22:00:00,0.00195,1.08736
1151,0.0,-9.8,94.7,0.7,82560.0,0.000156,0.40335,2018-01-22,23:00:00,0.00230,1.08697
